In [1]:
import numpy as np
import sklearn
import os
import string
import re
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

In [2]:
def convert_text(s):
    # Removes all characters from string except letters and digits and convert letters to lowercase
    return re.sub("[^a-zA-Z0-9]", " ", s.lower())

def read_txts(dir_path="./txt_sentoken/pos/"):
    # Reads all files from directory
    if dir_path[-1] != "/":
        dir_path = dir_path + "/"
    txt_list = []
    for file in os.listdir(dir_path):
        file = dir_path + file
        fin = open(file, 'r')
        txt = " ".join(fin.readlines())
        txt = convert_text(txt)
        txt_list.append(txt)
    return txt_list

In [3]:
class PoissonNB:
    def __init__(self, class_prior=None):
        """
        class_prior : np.array, size (n_classes,)
        Prior probabilities of the classes. If specified the priors are not
        adjusted according to the data.
        """
        self.class_prior = class_prior
    
    def fit(self, X, y, epsilon=1e-9):
        """
        Fit Poisson Naive Bayes according to X, y
        
        Parameters
        ----------
        X : np.array, shape (n_samples, n_features)
            Training vectors, where n_samples is the number of samples
            and n_features is the number of features.
        y : np.array, shape (n_samples,)
            Target values.
        """
        num_samples, num_features = X.shape
        self.classes = np.unique(y) 
        num_classes = self.classes.shape[0] 
        self.lambda_m = np.zeros((num_classes, num_features))
        self.class_prior = np.zeros(num_classes)
        for i, yi in enumerate(self.classes):
            Xi = X[y == yi, :]
            self.lambda_m[i,:] = np.mean(Xi, axis = 0) + epsilon
            self.class_prior[i] = float(Xi.shape[0]) / num_samples
        
    def predict(self, X):
        """
        Perform classification on an array of test vectors X.
        
        Parameters
        ----------
        X : np.array, shape = [n_samples, n_features]
        
        Returns
        -------
        C : np.array, shape = [n_samples]
            Predicted target values for X
        """
        probability_m = np.zeros((X.shape[0], self.classes.size))
        for i in range (self.classes.shape[0]):
            lg_class_prior = np.log(self.class_prior[i])
            prob_ij = np.sum(X*np.log(self.lambda_m[i,:]), axis = 1)
            prob_ij = prob_ij - np.sum(self.lambda_m[i,:])
            probability_m[:,i] = lg_class_prior + prob_ij
        return self.classes[np.argmax(probability_m, axis=1)]

1) Создадим два списка отзывов:

In [4]:
pos_feeds = read_txts("./txt_sentoken/pos/")
neg_feeds = read_txts("./txt_sentoken/neg/")

2.1) Создание обучающей выборки c вектором ответов:

In [5]:
pos_feeds , neg_feeds = shuffle(pos_feeds, neg_feeds)

In [6]:
learn_X = pos_feeds[:700] + neg_feeds[:700]
learn_Y = np.ones(1400)
learn_Y[700:] = 0
type(learn_X)

list

2.2) Создание контрольной выборки с вектором ответов:

In [7]:
test_X = pos_feeds[700:] + neg_feeds[700:]
test_Y = np.ones(600)
test_Y[300:] = 0

3) Создадим матрицу объект-признак:

In [8]:
vectorizer = CountVectorizer()

In [9]:
learn_X_tr = vectorizer.fit_transform(learn_X).toarray()
test_X_tr = vectorizer.transform(test_X).toarray()

5) Тест пуассоновного наивного байесовкого классификатора:

In [10]:
p_clf = PoissonNB()
p_clf.fit(learn_X_tr, learn_Y)
p_predict = p_clf.predict(test_X_tr)
print ("Accuracy is", np.mean(test_Y == p_predict))

Accuracy is 0.72


6.1) Тест гауссовского наивного байесовского классификатора

In [11]:
g_clf = GaussianNB()
g_clf.fit(learn_X_tr, learn_Y)
g_predict = g_clf.predict(test_X_tr)
print ("Accuracy is", np.mean(test_Y == g_predict))

Accuracy is 0.643333333333


6.2) Тест мультиномиального байесовского классификатора

In [12]:
m_clf = MultinomialNB()
m_clf.fit(learn_X_tr, learn_Y)
m_predict = m_clf.predict(test_X_tr)
print ("Accuracy is", np.mean(test_Y == m_predict))

Accuracy is 0.823333333333


7) Создаем функцию, которая принимает на вход строку с текстом рецензии, обученный классификатор, обученный объект класса CountVectorizer и печатает положительна ли данная рецензия.

In [13]:
def text_predict (text, clf, vectorizer):
    text = [convert_text(text)]
    text_tr = vectorizer.transform(text).toarray()
    if clf.predict(text_tr) == 1:
        print ('Positive feed')
    else: print ('Negative feed')

In [14]:
text = """after bloody clashes and independence won , lumumba refused to pander to the belgians , who continued a condescending and paternalistic relationship with the congo . 
their officers , particularly general janssens ( rudi delhem ) in the force publique , the congo's army , caused rebellions , undermining lumumba , who was outraged at the rape and murder of belgian nationals . 
with unrest building , moise tshombe ( pascal nzonzi ) and the province of katanga , which contained 70 percent of the country's resources , proclaimed secession . 
lumumba replaced janssens , making mobutu a colonel , and went on a pacification 
tour with congolese president joseph kasa vubu ( maka kotto ) , but it was too late . 
'when you want to drown a dog , you say it has rabies , ' prophesies lumumba of his own fate . 
peck and bonitzer do an exemplary job telling a complicated tale with a myriad of players , although they frequently succumb to cliche , particularly regarding lumumba's private life . 
peck's script illuminates bantu sayings like 'the hand that gives , rules' when lumumba uses it with the american ambassador . 
peck's direction is less assured , with many scenes unfortunately playing like standard television fare . 
he's served well , though , by his casting of ebouaney in the title role . 
ebouaney is dynamic , radiating his character's fierce passion for his people and his country . 
lumumba's intelligence and ability to strategize , even as he's cornered by insurmountable odds , are given life by ebouaney . 
peck's subject and lead actor elevate his film above its mediocre production . 
 " lumumba " is a story that deserves to be told and ebouaney's performance makes the tragedy personally felt . """

In [15]:
text_predict (text, m_clf, vectorizer)

Positive feed


8) Почему наивный байесовский классификатор плохо или хорошо работает для данной задачи?

Наивный байесовский классификатор хорошо подходит для задач классификации данных. Основным его допущением является независимость одного признакого от другого, в нашем случае это означает, что появление одного слова никак не зависит от другого. С одной стороны кажется, что совершать такое допущение является большой ошибкой, однако результаты предсказаний данного классификатора говорят об обратном. Таким образом это превращается в один из его плюсов. Так как время необходимое для обучения является достотачно небольшим, к тому же это хорошо подходит для небольшой выборки обучающих данных. 

Бонус 2) Была найдена выборка "Reuters-21578 dataset" в которой 5485 текстов разделенных по 8 категориям. Источник http://ana.cachopo.org/datasets-for-single-label-text-categorization

Осуществим чтение файла с созданием выборки и вектора ответов:

In [16]:
data_X = []
data_Y = []
with open("r8-train-all-terms.txt") as f:
    for line in f:
        line = re.split(r'\t+', line)
        txt = "".join(line[1:])
        data_Y.append(line[0]) # достаем класс, обозначенный первым словом
        data_X.append(txt) # формируем тексты без первого слова, указывающее класс

Случайным образом выберем 3000 текстов для обучающей выборки и оставшиеся 2485 для теста:

In [17]:
data_X, data_Y = shuffle(data_X, data_Y)
learn_data_X = data_X[:3000]
learn_data_Y = data_Y[:3000]
test_data_X = data_X[3000:]
test_data_Y = data_Y[3000:]

Создадим матрицу "объект - признак":

In [18]:
vectorizer = CountVectorizer()
learn_data_X_tr = vectorizer.fit_transform(learn_data_X).toarray()
test_data_X_tr = vectorizer.transform(test_data_X).toarray()

Применим различные классификаторы и посмотрим их точность:

In [19]:
g_clf = GaussianNB()
g_clf.fit(learn_data_X_tr, learn_data_Y)
g_predict = g_clf.predict(test_data_X_tr)

In [20]:
m_clf = MultinomialNB()
m_clf.fit(learn_data_X_tr, learn_data_Y)
m_predict = m_clf.predict(test_data_X_tr)

In [21]:
print ("GaussianNB accuracy is", np.mean(test_data_Y == g_predict))
print ("MultinomialNB accuracy is", np.mean(test_data_Y == m_predict))

GaussianNB accuracy is 0.819315895372
MultinomialNB accuracy is 0.920724346076


In [22]:
p_clf = PoissonNB()
p_clf.fit(learn_data_X_tr, learn_data_Y)
p_predict = p_clf.predict(test_data_X_tr)
print ("PoissonNB accuracy is", np.mean(test_data_Y == p_predict))

PoissonNB accuracy is 0.280482897384


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:28: DeprecationWarning: using a boolean instead of an integer will result in an error in the future
